# Исследование в отсеивании признаков
Планируется отсеивать, признаки:
* высокий процент пропущенных значений;
* колинеальность (сильная корреляция);
* нулевая важность в моделях, основанных на деревьях;
* низкая важность;
* единственное значение

Данный ноутбук написан после прочтения http://proglib.io/p/feature-selector

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/agents2020.csv')
df.describe()

,Наименование ДП,Итого,"Макс. ПДЗ за 2020 год, дней","Сред. ПДЗ за 2020 год, дней","Кол-во просрочек свыше 5-ти дней за 2020 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.","Кол-во раз ПДЗ за 2020 год, шт.","2016, Нематериальные активы, RUB","2017, Нематериальные активы, RUB","2018, Нематериальные активы, RUB",...,"2018, Себестоимость продаж, RUB","2019, Себестоимость продаж, RUB","2016, Прибыль (убыток) до налогообложения , RUB","2017, Прибыль (убыток) до налогообложения , RUB","2018, Прибыль (убыток) до налогообложения , RUB","2019, Прибыль (убыток) до налогообложения , RUB","2016, Прибыль (убыток) от продажи, RUB","2017, Прибыль (убыток) от продажи, RUB","2018, Прибыль (убыток) от продажи, RUB","2019, Прибыль (убыток) от продажи, RUB"
count,531.000000,525.000000,531.000000,531.000000,531.000000,5.310000e+02,531.000000,5.310000e+02,5.310000e+02,5.310000e+02,...,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02,5.310000e+02
mean,265.000000,30.400000,26.924670,10.263054,9.167608,6.155193e+06,22.875706,2.581892e+07,3.600098e+07,4.672202e+07,...,4.890154e+09,5.672858e+09,1.395727e+09,1.075368e+09,1.312543e+09,2.561710e+09,1.054666e+09,1.206979e+09,1.988515e+09,2.262094e+09
std,153.430766,48.575431,69.438625,32.093828,31.945180,2.252694e+07,54.068576,1.933860e+08,3.142319e+08,3.658906e+08,...,1.852615e+10,2.113821e+10,9.907869e+09,8.439904e+09,1.193399e+10,2.092975e+10,7.361247e+09,8.196642e+09,1.271200e+10,1.727994e+10
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,-4.359633e+09,-4.917329e+10,-1.316305e+11,-1.482809e+10,-1.126443e+09,-9.205369e+08,-2.520849e+10,-5.564792e+09
25%,132.500000,10.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,1.573535e+08,1.916417e+08,4.458599e+04,2.038217e+04,8.356688e+05,1.415924e+06,0.000000e+00,5.796178e+04,1.482803e+06,1.636306e+06
50%,265.000000,20.000000,5.000000,2.673077,0.000000,0.000000e+00,3.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,4.755548e+08,5.199580e+08,3.459236e+06,4.535032e+06,6.507006e+06,9.692994e+06,5.703185e+06,8.603822e+06,1.306624e+07,1.458471e+07
75%,397.500000,30.000000,18.000000,6.049462,6.000000,2.739224e+06,21.000000,9.235669e+03,1.019108e+04,2.707006e+04,...,1.709260e+09,1.849968e+09,4.339904e+07,4.402580e+07,5.766210e+07,6.935955e+07,5.960605e+07,7.043376e+07,8.941911e+07,9.462261e+07
max,530.000000,370.000000,489.000000,357.000000,445.000000,2.716331e+08,478.000000,3.126233e+09,6.320583e+09,7.549618e+09,...,2.111127e+11,2.224682e+11,1.237635e+11,1.102105e+11,1.349557e+11,3.983158e+11,1.393573e+11,1.483514e+11,2.091451e+11,3.420850e+11


In [8]:
print(f'В DF всего {df.columns.shape[0]} признаков.')
target_feature = 'Макс. ПДЗ за 2020 год, дней'
print(f'Целевой признак "{target_feature}"')
target_year = '2020'
new_column = [] # Прзнаки для фильтрации
for column in df.columns:
    if target_year not in column:
        new_column.append(column)
print(f'После фильтрации осталось {len(new_column)} признаков.')

В DF всего 131 признаков.
Целевой признак "Макс. ПДЗ за 2020 год, дней"
После фильтрации осталось 126 признаков.


In [9]:
columns_del = [] # Список признаков, которые надо удалить из данных
# Считаю кореляцию значений
df_corr = df[new_column].corr()

In [36]:
import itertools
# Нахожу все сильно коррелирующие признаки
def get_corr(df, corr_max=0.98):
    rez = []
    columns_comb = itertools.combinations(df.columns,2)
    for column, index in columns_comb:
        if df.loc[column, index] > corr_max:
            rez.append((column, index))
    return rez

corr = get_corr(df_corr)
print(f' Найдено {len(corr)} сильно коррелирующих пар признаков')
print(*corr, sep='\n')

# Добавляю найденные признаки в список на удаление
# Добавляю левое значение т.к. там скорее дата меньше
for i in corr:
    if corr[0] not in columns_del:
        columns_del.append(corr[0])

31
('2016, Основные средства , RUB', '2017, Основные средства , RUB')
('2016, Основные средства , RUB', '2018, Основные средства , RUB')
('2016, Основные средства , RUB', '2019, Основные средства , RUB')
('2016, Основные средства , RUB', '2017, Внеоборотные активы, RUB')
('2017, Основные средства , RUB', '2018, Основные средства , RUB')
('2017, Основные средства , RUB', '2019, Основные средства , RUB')
('2017, Основные средства , RUB', '2017, Внеоборотные активы, RUB')
('2018, Основные средства , RUB', '2019, Основные средства , RUB')
('2018, Основные средства , RUB', '2017, Внеоборотные активы, RUB')
('2016, Внеоборотные активы, RUB', '2018, Внеоборотные активы, RUB')
('2016, Внеоборотные активы, RUB', '2019, Внеоборотные активы, RUB')
('2018, Внеоборотные активы, RUB', '2019, Внеоборотные активы, RUB')
('2016, Уставный капитал , RUB', '2017, Уставный капитал , RUB')
('2016, Уставный капитал , RUB', '2018, Уставный капитал , RUB')
('2016, Уставный капитал , RUB', '2019, Уставный капит